In [25]:
import pandas as pd

X_train = pd.read_csv('https://raw.githubusercontent.com/mkleinbort/Kaggle-COMPAS/main/train/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/mkleinbort/Kaggle-COMPAS/main/train/y_train.csv', squeeze=True)

X_test = pd.read_csv('https://raw.githubusercontent.com/mkleinbort/Kaggle-COMPAS/main/test/X_test.csv')

In [26]:
X_train.head()

,id,name,first,last,sex,date_of_birth,age,age_group,race,juv_fel_count,...,c_charge_desc,r_jail_in,r_jail_out,type_of_assessment,screening_date,v_type_of_assessment,v_screening_date,in_custody,out_custody,start
0,2677,nicholas staggewise,nicholas,staggewise,Male,1994-02-28,22,Less than 25,Caucasian,0,...,arrest case no charge,NaN,NaN,Risk of Recidivism,2013-09-24,Risk of Violence,2013-09-24,2013-09-23,2013-09-27,3
1,548,newton smith,newton,smith,Male,1995-01-28,21,Less than 25,African-American,0,...,Grand Theft in the 3rd Degree,NaN,NaN,Risk of Recidivism,2013-06-17,Risk of Violence,2013-06-17,2013-06-16,2013-06-16,0
2,4483,terrance irvin,terrance,irvin,Male,1984-04-10,32,25 - 45,African-American,0,...,Grand Theft in the 3rd Degree,2013-12-21,2014-02-21,Risk of Recidivism,2013-10-12,Risk of Violence,2013-10-12,2013-10-11,2013-10-15,3
3,9476,jaimie peterson,jaimie,peterson,Female,1986-01-02,30,25 - 45,African-American,0,...,Felony Driving While Lic Suspd,NaN,NaN,Risk of Recidivism,2014-01-12,Risk of Violence,2014-01-12,2014-01-11,2014-01-12,0
4,199,ritesh sukhlall,ritesh,sukhlall,Male,1987-05-12,28,25 - 45,Caucasian,0,...,arrest case no charge,NaN,NaN,Risk of Recidivism,2013-08-29,Risk of Violence,2013-08-29,NaN,NaN,0


In [27]:
X_test.head()

,id,name,first,last,sex,date_of_birth,age,age_group,race,juv_fel_count,...,c_charge_desc,r_jail_in,r_jail_out,type_of_assessment,screening_date,v_type_of_assessment,v_screening_date,in_custody,out_custody,start
0,10257,wendy saintjuste,wendy,saintjuste,Female,1982-11-10,33,25 - 45,Other,0,...,Grand Theft in the 3rd Degree,NaN,NaN,Risk of Recidivism,2013-11-19,Risk of Violence,2013-11-19,2013-11-18,2013-11-19,0
1,468,seymour brown,seymour,brown,Male,1988-04-23,27,25 - 45,African-American,0,...,Aggrav Battery w/Deadly Weapon,NaN,NaN,Risk of Recidivism,2014-01-14,Risk of Violence,2014-01-14,2014-01-14,2014-01-31,17
2,2208,mario baptiste,mario,baptiste,Male,1984-02-09,32,25 - 45,African-American,0,...,Driving While License Revoked,2014-06-26,2014-06-28,Risk of Recidivism,2013-08-26,Risk of Violence,2013-08-26,2014-06-26,2014-06-28,1
3,5351,colter thompson,colter,thompson,Male,1994-07-15,21,Less than 25,Caucasian,0,...,Possess Cannabis/20 Grams Or Less,NaN,NaN,Risk of Recidivism,2013-04-09,Risk of Violence,2013-04-09,2013-04-08,2013-04-09,0
4,4143,eric floyd,eric,floyd,Male,1967-07-12,48,Greater than 45,African-American,0,...,Felony Battery (Dom Strang),NaN,NaN,Risk of Recidivism,2013-12-28,Risk of Violence,2013-12-28,2013-12-28,2013-12-30,2


In [28]:
from catboost import CatBoostClassifier

def get_cat_columns(df):
    return [i for i, dtype in enumerate(df.dtypes) if dtype in [object]]

cb = CatBoostClassifier(cat_features=get_cat_columns(pd.concat([X_train, X_test])), 
                        metric_period=50, n_estimators=200)

In [29]:
cb.fit(X_train.fillna(-1)
       , y_train)

Learning rate set to 0.329979
0:	learn: 0.6797953	total: 19.6ms	remaining: 3.91s
50:	learn: 0.2077083	total: 2.01s	remaining: 5.89s
100:	learn: 0.1633804	total: 3.92s	remaining: 3.84s
150:	learn: 0.1332003	total: 5.82s	remaining: 1.89s
199:	learn: 0.1067604	total: 7.62s	remaining: 0us


In [30]:
y_pred = cb.predict_proba(X_test.fillna(-1))

In [31]:
submission = pd.DataFrame(y_pred, columns=cb.classes_, index=X_test.index)

In [32]:
submission.to_csv('submission.csv', index=False)

In [33]:
x = pd.Series(cb.feature_importances_, index=X_train.columns).sort_values()

In [34]:
x

c_jail_in                   0.000000
name                        0.000000
v_type_of_assessment        0.000000
type_of_assessment          0.000000
juv_fel_count               0.135765
c_jail_out                  0.524338
date_of_birth               0.586057
juv_other_count             0.815792
c_charge_degree             0.846894
juv_misd_count              0.851105
sex                         1.450373
r_jail_in                   1.726181
c_arrest_date               1.767170
days_b_screening_arrest     1.848911
age_group                   2.813889
race                        2.895042
last                        3.091697
start                       3.106570
first                       3.350312
screening_date              3.636160
in_custody                  3.775585
out_custody                 4.335876
age                         4.881146
id                          5.237085
priors_count                6.005297
c_offense_date              6.005720
c_charge_desc               7.066114
v

In [45]:
a = X_train[['r_jail_in','r_jail_out']].assign(y=y_train).assign(d = lambda x: x['r_jail_out'].isna())

In [54]:
a[['y','d']].pivot_table(index='y', values='d', aggfunc='count')

,d
y,
No-Recidivism,2997
Non-Violent,649
Violent,148
